In [1]:
import numpy as np
from symtdd import ts, tn

In [2]:
from symtdd.io.qiskit_import import cir_2_tn
from symtdd import IndexType, TensorType, TDDWeightType

In [3]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter

In [4]:
qa = QuantumRegister(2, 'qa')
qb = QuantumRegister(3, 'qb')
# va = Parameter('x0')
cir = QuantumCircuit(qa, qb)
# cir.rz(va, 3)
cir.cx(1, 4)
cir.cy(1, 4)
cir.ccx(1, 3, 4)
cir.h(2)

In [5]:
tna = cir_2_tn(cir, IndexType.STANDARD, TensorType.TENSOR, TDDWeightType.NA)

In [6]:
# tna = cir_2_tn(cir, IndexType.STANDARD, TensorType.TDD, TDDWeightType.COMPLEX)

In [7]:
# tna.tensors[-1].show()

In [8]:
tsa = tna.contract()

DEBUG:symtdd.ts:====Contract
DEBUG:symtdd.ts:(y_0_0#0,),(x_1_0#0,)->[y_0_0#0, x_1_0#0]
DEBUG:symtdd.ts:
input: TS None: shape (2,), indices ['y_0_0#0']
output: (0,)
DEBUG:symtdd.ts:
input: TS None: shape (2,), indices ['x_1_0#0']
output: (1,)
DEBUG:symtdd.ts:data shape: (2, 2)
DEBUG:symtdd.ts:Contract End====
DEBUG:symtdd.ts:====Contract
DEBUG:symtdd.ts:(x_2_0#0,),(y_0_0#0, x_1_0#0)->[y_0_0#0, x_1_0#0, x_2_0#0]
DEBUG:symtdd.ts:
input: TS None: shape (2,), indices ['x_2_0#0']
output: (2,)
DEBUG:symtdd.ts:
input: TS None: shape (2, 2), indices ['y_0_0#0', 'x_1_0#0']
output: (0, 1)
DEBUG:symtdd.ts:data shape: (2, 2, 2)
DEBUG:symtdd.ts:Contract End====
DEBUG:symtdd.ts:====Contract
DEBUG:symtdd.ts:(x_3_0#0,),(y_0_0#0, x_1_0#0, x_2_0#0)->[y_0_0#0, x_1_0#0, x_3_0#0, x_2_0#0]
DEBUG:symtdd.ts:
input: TS None: shape (2,), indices ['x_3_0#0']
output: (2,)
DEBUG:symtdd.ts:
input: TS None: shape (2, 2, 2), indices ['y_0_0#0', 'x_1_0#0', 'x_2_0#0']
output: (0, 1, 3)
DEBUG:symtdd.ts:data shape: (2, 2

In [9]:
tsa.sort()
for index in tsa.indices:
    print(index)

y_0_0_0
y_1_0_0
y_2_0_0
y_3_0_0
y_4_0_0


In [10]:
tsa.str_hyprindices

['y_0_0#0', 'y_1_0#0', 'y_2_0#0', 'y_3_0#0', 'y_4_0#0']

In [11]:
import numpy as np
ordered_tsa_data = tsa.data
ordered_tsa_data.flatten()

array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j])

In [12]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')
job = backend.run(cir)
result = job.result()
result.get_statevector()

Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j],
            dims=(2, 2, 2, 2, 2))


In [13]:
np.testing.assert_allclose(ordered_tsa_data.flatten(), result.get_statevector())

In [14]:
from qiskit.circuit.random import random_circuit
cir = random_circuit(2, 2, measure=False, seed=12345)
cir2 = cir.copy()
cir.save_statevector()
backend = Aer.get_backend('aer_simulator_statevector')
job = backend.run(cir)
result = job.result()
gold_data = result.get_statevector()

In [15]:
cir.draw()

┌──────────────────────────┐ statevector 
q_0: ────────────■────────────┤ U3(3.7593,1.1733,4.2271) ├──────░──────
     ┌───────────┴───────────┐└──────┬───────────┬───────┘      ░      
q_1: ┤ U3(1.9903,5.01,4.249) ├───────┤ P(5.9175) ├──────────────░──────
     └───────────────────────┘       └───────────┘              ░

In [16]:
gold_data.data

array([-0.3039595 +0.j       ,  0.36880765+0.8784017j,
        0.        +0.j       ,  0.        +0.j       ])

In [17]:
tn = cir_2_tn(cir2)
ts = tn.contract()
ts.sort()

TypeError: cir_2_tn() missing 2 required positional arguments: 'idxtype' and 'tstype'

In [ ]:
print(ts, ts.data)

TS None: shape (2, 2), indices ['y_0_0#0', 'y_1_0#0'] [[-0.3039595 +0.j         0.        +0.j       ]
 [ 0.36880765+0.8784017j  0.        +0.j       ]]


In [ ]:
class Abase():
    def __init__(self):
        print("Abase init")
        super().__init__()
        print(self.__class__, 'a')

class Bbase():
    def __init__(self):
        print("Bbase init")
        super().__init__()
        print(self.__class__, 'b')

class AB(Abase, Bbase):
    def __init__(self):
        super().__init__()
        print('c')

aabb = AB()

Abase init
Bbase init
<class '__main__.AB'> b
<class '__main__.AB'> a
c
